In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import cv2
import random
import numpy as np
from google.colab.patches import cv2_imshow
import zipfile

In [ ]:
def overlay_transparent(background, overlay, x, y):
    h, w = overlay.shape[0], overlay.shape[1]

    overlay_image = overlay[..., :3]   # First 3 Channel BGR is used as overlay
    mask = overlay[..., 3:] / 255.0    # Fourth channel of overlay is used as mask

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

def overlay_mask(background, overlay, x, y):
    h, w = overlay.shape[0], overlay.shape[1]
    background[y:y+h, x:x+w] = overlay
    return background

In [ ]:
basePath = "/content/gdrive/My Drive/EVA/S15/images"
os.chdir(basePath)

In [ ]:
fg_bg_zip = zipfile.ZipFile('fg_bg_1.zip', mode='a', compression=zipfile.ZIP_STORED)
fg_bg_mask_zip = zipfile.ZipFile('fg_bg_mask_1.zip', mode='a', compression=zipfile.ZIP_STORED)

In [ ]:
count = 1
for i in range(1, 101):  #bg loop
    print("Processing Bg ===== ", i, " ==== count ==== ", count )
    bgImgPath = basePath+"/background/"+str(i)+".jpg"
    bgImg = cv2.imread(bgImgPath, cv2.IMREAD_UNCHANGED)
    bgHeight = bgImg.shape[0]
    bgWidth = bgImg.shape[1]
    for j in range(1, 101): #fg loop
        fgImgPath = basePath+"/foreground/"+str(j)+".png"
        fgMaskPath = basePath+"/fg_mask/"+str(j)+".jpg"
        fgImg = cv2.imread(fgImgPath, cv2.IMREAD_UNCHANGED)
        fgMask = cv2.imread(fgMaskPath, cv2.IMREAD_UNCHANGED)
        fgImgFlip = cv2.flip(fgImg, 1)
        fgMaskFlip = cv2.flip(fgMask, 1)
        fgImgs = [fgImg, fgImgFlip]
        fgMasks = [fgMask, fgMaskFlip]
        for p in range(len(fgImgs)):
            for k in range(20):
                bgImgCopy = bgImg.copy()
                coords = random.sample(range(0, 112), 2)
                fgBg = overlay_transparent(bgImgCopy, fgImgs[p], coords[0], coords[1])
                bgMask = np.zeros((bgHeight, bgWidth), dtype=fgMask.dtype)
                fgBgMask = overlay_mask(bgMask, fgMasks[p], coords[0], coords[1])
                cv2.imwrite("fg_bg_tmp.jpg", fgBg, [cv2.IMWRITE_JPEG_QUALITY, 40])
                cv2.imwrite("fg_bg_mask.jpg", fgBgMask, [cv2.IMWRITE_JPEG_QUALITY, 35])
                fg_bg_zip.write('fg_bg_tmp.jpg', f'bg_{i:03d}/{count}.jpg')
                fg_bg_mask_zip.write('fg_bg_mask.jpg', f'bg_{i:03d}/{count}.jpg')
                # cv2.imwrite(basePath+"/fg_bg/"+str(count)+".jpg", fgBg)
                # cv2.imwrite(basePath+"/fg_bg_mask/"+str(count)+".jpg", fgBgMask)
                count += 1


Processing Bg =====  1  ==== count ====  1
Processing Bg =====  2  ==== count ====  4001
Processing Bg =====  3  ==== count ====  8001
Processing Bg =====  4  ==== count ====  12001
Processing Bg =====  5  ==== count ====  16001
Processing Bg =====  6  ==== count ====  20001
Processing Bg =====  7  ==== count ====  24001
Processing Bg =====  8  ==== count ====  28001
Processing Bg =====  9  ==== count ====  32001
Processing Bg =====  10  ==== count ====  36001
Processing Bg =====  11  ==== count ====  40001
Processing Bg =====  12  ==== count ====  44001
Processing Bg =====  13  ==== count ====  48001
Processing Bg =====  14  ==== count ====  52001
Processing Bg =====  15  ==== count ====  56001
Processing Bg =====  16  ==== count ====  60001
Processing Bg =====  17  ==== count ====  64001
Processing Bg =====  18  ==== count ====  68001
Processing Bg =====  19  ==== count ====  72001
Processing Bg =====  20  ==== count ====  76001
Processing Bg =====  21  ==== count ====  80001
Process

In [ ]:
fg_bg_mask_zip.close()
fg_bg_zip.close()